<a href="https://colab.research.google.com/github/Youmei-luo/ESG_NLP/blob/main/esg_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!python -m spacy download en_core_web_sm
!python -m spacy download ja_core_news_sm

2022-11-13 07:00:18.870299: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 4.6 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2022-11-13 07:00:30.369112: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 3.4 MB/s 
     |████████████████████████████████| 2.2 MB 5.0 MB/s 
  Created wheel for sudachidict-core: filename=SudachiDict_core-20221021-py3-none-any.whl size=71574781 sha256=473cd6846d22fa1e162f56d2b6256adcf1fe2c78fe24da0fccad2ccade811566
  Stored in directory: /root

In [53]:
# ファイルはdata/raw/のディレクトリに配置してください

# file_name = "2021_001_integrated_en.pdf"
# file_name = "2019_001_annual_en.pdf"
file_name = "2021_001_integrated_jp.pdf"

In [54]:
# LANG = "en"
LANG = "ja"

In [ ]:
pip install pdfminer.six

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 4.6 MB/s 
     |████████████████████████████████| 4.1 MB 35.3 MB/s 


In [55]:
import unicodedata
from io import StringIO
from pdfminer.high_level import extract_text, extract_text_to_fp
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import HTMLConverter, TextConverter
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd


class PDFReader():

    def __init__(self):
        pass

    def read(self, path, html=False):
        text = StringIO()
        if html:
            with open(path, "rb") as f:
                extract_text_to_fp(f, text, laparams=LAParams(),
                                   output_type="html", codec=None)
            text = text.getvalue()
        else:
            text = extract_text(path)
        return text

    def read_pages(self, path, html=False,
                   laparams=None, maxpages=0, page_numbers=None,
                   password="", scale=1.0, rotation=0, layoutmode='normal',
                   output_dir=None, strip_control=False, debug=False,
                   disable_caching=False, **kwargs):

        rsrcmgr = PDFResourceManager(caching=True)
        pages = []
        with open(path, "rb") as f:
            for page in PDFPage.get_pages(f, None, maxpages=0,
                                          check_extractable=True):
                page.rotate = (page.rotate + rotation) % 360
                text = StringIO()
                if html:
                    device = HTMLConverter(rsrcmgr, text, codec=None, scale=scale,
                                           layoutmode=layoutmode, laparams=laparams)
                else:
                    device = TextConverter(rsrcmgr, text, codec=None, laparams=laparams)

                interpreter = PDFPageInterpreter(rsrcmgr, device)
                interpreter.process_page(page)
                pages.append(text.getvalue())
                device.close()

        return pages

    def read_to_frame(self, path):
        items = []
        pages = self.read_pages(path, html=True)
        page_index = 0
        for i, p in enumerate(pages):
            p = p.replace("<br>", "\n").replace("<br/>", "\n")
            html = BeautifulSoup(p, "html.parser")
            if not html:
                continue
            page = html.get_text("\n")
            contents = page.split("\n\n")
            content_index = 0
            for j, s in enumerate(contents):
                c = s.strip()
                if not c:
                    continue

                item = {
                    "page": page_index,
                    "order": content_index,
                    "content": c
                }
                items.append(item)
                content_index += 1

            page_index += 1

        df = pd.DataFrame(items)
        return df

    def normalize(self, text):
        if text is None:
            return ""
        _text = text.replace("\r", "").replace("\n", "").strip()
        _text = unicodedata.normalize("NFKC", _text)
        return _text

    def preprocess_frame(self, df, lower=True):
        repeat_check = []
        preprocessed = []

        for i, row in df.iterrows():
            content = self.normalize(row["content"])
            if lower:
                content = content.lower()
            if content in repeat_check:
                continue
            repeat_check.append(content)

            item = {}
            for c in df.columns:
                item[c] = row[c]
                if c == "content":
                    item[c] = content

            preprocessed.append(item)

        df = pd.DataFrame(preprocessed)
        return df


In [56]:
import os, sys
# sys.path.append("../")
from pathlib import Path
import pandas as pd
import numpy as np
# from esg_nlp.data.pdf_reader import PDFReader


# path = f"../data/raw/{file_name}"
path = f"/content/drive/MyDrive/Colab Notebooks/data/raw/{file_name}"
reader = PDFReader()
df = reader.read_to_frame(path)

In [57]:
df.head(5)

,page,order,content
0,0,0,Page 1
1,0,1,Integrated Report \n2021\n統合報告書\n2021\n2021\n年...
2,1,0,Page 1
3,1,1,TOYOTA MOTOR CORPORATION\n INTEGRATED REPORT\n13
4,1,2,社長メッセージ


In [58]:
import unicodedata


def preprocess(text):
    if text is None:
        return ""
    _text = text
    _text = unicodedata.normalize("NFKC", _text)  #半角/全角の統一
    _text = _text.replace("\r", "").replace("\n", " ")  # 文中改行の削除
    _text = _text.lower()
    return _text


df["preprocessed"] = df.content.apply(preprocess)

In [59]:
preprocessed_df = df.drop_duplicates(subset=["preprocessed"])

In [60]:
limit_length = 100

In [61]:
preprocessed_df = preprocessed_df.assign(length=preprocessed_df["preprocessed"].apply(lambda s: len(s)))
preprocessed_df = preprocessed_df[preprocessed_df["length"] > limit_length]

In [62]:
print(f"Rows are decreased from {len(df)} to {len(preprocessed_df)}")

Rows are decreased from 1317 to 189


In [63]:
preprocessed_df.head(10)

,page,order,content,preprocessed,length
9,1,7,カーボンニュートラル実現に向けた取り組み：バッテリー\nEV\n戦略\nトヨタは\n2021...,カーボンニュートラル実現に向けた取り組み:バッテリー ev 戦略 トヨタは 2021 年 1...,337
10,1,8,カーボンニュートラル実現へトヨタがすべきこと\nオープニング映像が終わり、会場が明かりで照ら...,カーボンニュートラル実現へトヨタがすべきこと オープニング映像が終わり、会場が明かりで照らさ...,324
11,1,9,限りすぐに、\nCO\n2\nを減らさなければなりません。\n今、私たちは、多様化した世界で...,限りすぐに、 co 2 を減らさなければなりません。 今、私たちは、多様化した世界で、何が正...,364
12,1,10,ベールを脱いだ５台の\nbZ\n豊田\n本日、皆様に、私たちが準備している新型車をご披露させ...,ベールを脱いだ5台の bz 豊田 本日、皆様に、私たちが準備している新型車をご披露させていた...,658
13,1,11,航続距離を伸ばすために電池を増やせば、クルマは大きく、重く、高くなります。小さいクルマだから...,航続距離を伸ばすために電池を増やせば、クルマは大きく、重く、高くなります。小さいクルマだから...,359
26,1,24,対象期間\n2020\n年度（\n2020\n年\n4\n月〜\n2021\n年\n3\n月...,対象期間 2020 年度( 2020 年 4 月〜 2021 年 3 月) 一部 202...,227
27,1,25,2\n \n社長メッセージ\n5\n \n価値創造の源流：トヨタらしさ\n5\n 創業の精神...,2 社長メッセージ 5 価値創造の源流:トヨタらしさ 5 創業の精神 6 ...,518
28,1,26,本統合報告書\n2021\n（\nIntegrated Report 2021\n）は、トヨ...,本統合報告書 2021 ( integrated report 2021 )は、トヨタがめざ...,180
39,1,37,報告書／レポート体系\n（注）トヨタ公式企業サイトでは、上記報告書以外の取り組みの情報も掲載...,報告書/レポート体系 (注)トヨタ公式企業サイトでは、上記報告書以外の取り組みの情報も掲載し...,181
54,2,8,社長就任以来\n12\n年間の振り返り\n今もなお、世界中で、新型コロナウイルスの影響は続い...,社長就任以来 12 年間の振り返り 今もなお、世界中で、新型コロナウイルスの影響は続いており...,885


In [64]:
if LANG == "ja":
    question = "組織が特定した、短期・中期・長期の気候関連のリスクと機会を記述する。"    
else:
    question = "Describe the climate-related risks and opportunities the organization has identified over the short, medium, and long term."
question = question.lower()

In [24]:
import re
from spacy.util import get_lang_class


class Parser():

    def __init__(self, lang):
        self.lang = lang
        self.parser = get_lang_class(self.lang)()
    
    def parse(self, text):
        return self.parser(text)

In [65]:
parser = Parser(LANG)
question_words = [str(t).strip() for t in parser.parse(question) if not t.is_stop and not re.match("\'|\.|\?|\/|\,|\-|、|・|。", t.text)]
question_words

['組織', '特定', '短期', '中期', '長期', '気候', '関連', 'リスク', '機会', '記述']

In [66]:
def count_keyword_match(text, keywords):
    count = 0
    for k in keywords:
        if k in text:
            count += 1
    return count


preprocessed_df = preprocessed_df.assign(
    keyword_match=preprocessed_df["preprocessed"].apply(
        lambda s: count_keyword_match(s, question_words))
)

In [67]:
matched_df = preprocessed_df[preprocessed_df["keyword_match"] > 0]
matched_df.sort_values(by=["keyword_match"], ascending=False).head(5)

,page,order,content,preprocessed,length,keyword_match
780,35,18,トヨタは、金融安定理事会「気候関連財務情報\n開示タスクフォース（\nTCFD\n）」の提言...,トヨタは、金融安定理事会「気候関連財務情報 開示タスクフォース( tcfd )」の提言に 2...,591,6
781,35,19,戦略\nトヨタ環境チャレンジ\n2050\n トヨタは気候変動、水不足、資源枯渇、生物多様性...,戦略 トヨタ環境チャレンジ 2050 トヨタは気候変動、水不足、資源枯渇、生物多様性低下な...,1093,4
782,35,20,重要度の高いリスク／機会と対応\nリスク機会トヨタの対応気候シナリオとの関係現行シナリオ対策...,重要度の高いリスク/機会と対応 リスク機会トヨタの対応気候シナリオとの関係現行シナリオ対策強...,622,3
814,36,17,販車も含めた\nCO\n2\n排出削減に向け、技術の選択肢\nを広げることも重要であると考え...,販車も含めた co 2 排出削減に向け、技術の選択肢 を広げることも重要であると考えています...,845,3
27,1,25,2\n \n社長メッセージ\n5\n \n価値創造の源流：トヨタらしさ\n5\n 創業の精神...,2 社長メッセージ 5 価値創造の源流:トヨタらしさ 5 創業の精神 6 ...,518,2


In [68]:
sentences = []
for i, row in preprocessed_df.iterrows():
    c = row["preprocessed"]
    for j, s in enumerate(c.replace("•", ".").replace(";", ".").split(".")):
        sentences.append({
            "page": row["page"],
            "order": row["order"],
            "sentence_order": j,
            "sentence": s[:512],
            "length": len(s)
        })

sentences_df = pd.DataFrame(sentences)
sentences_df.head(5)

,page,order,sentence_order,sentence,length
0,1,7,0,カーボンニュートラル実現に向けた取り組み:バッテリー ev 戦略 トヨタは 2021 年 1...,337
1,1,8,0,カーボンニュートラル実現へトヨタがすべきこと オープニング映像が終わり、会場が明かりで照らさ...,324
2,1,9,0,限りすぐに、 co 2 を減らさなければなりません。 今、私たちは、多様化した世界で、何が正...,364
3,1,10,0,ベールを脱いだ5台の bz 豊田 本日、皆様に、私たちが準備している新型車をご披露させていた...,658
4,1,11,0,航続距離を伸ばすために電池を増やせば、クルマは大きく、重く、高くなります。小さいクルマだから...,359


In [69]:
len(sentences_df)

462

In [33]:
!git clone https://github.com/huggingface/transformers
%cd transformers
!pip install .

Cloning into 'transformers'...
remote: Enumerating objects: 115030, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 115030 (delta 41), reused 51 (delta 24), pack-reused 114945
Receiving objects: 100% (115030/115030), 108.42 MiB | 23.78 MiB/s, done.
Resolving deltas: 100% (85604/85604), done.
/content/transformers/transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/transformers/transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements 

In [38]:
%pip install fugashi
%pip install ipadic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 583 kB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.4 MB 5.2 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556722 sha256=1d580b412f2ed8b48c18f51cc1d1834c4a2f0e0193076e02b47e75e562a48d17
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic


In [40]:
import numpy as np
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer
from transformers.pipelines import FeatureExtractionPipeline
from sklearn.metrics.pairwise import cosine_similarity


class DistilBertFeatureExtractionPipeline(FeatureExtractionPipeline):
    
    def preprocess(self, inputs, truncation=True):
        # Make truncation=True
        # https://huggingface.co/docs/transformers/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.__call__.truncation
        return_tensors = self.framework
        if truncation is None:
            kwargs = {}
        else:
            kwargs = {"truncation": truncation}
        # Refer the issue
        # https://github.com/huggingface/transformers/issues/2702
        model_inputs = self.tokenizer(inputs, return_token_type_ids=False,
                                      return_tensors=return_tensors, **kwargs)
        return model_inputs


def encode(pretrained_model_name, text, batch_size=10, pretrained_tokenizer_name=""):
    print("Loading pretrained model...")
    model = AutoModel.from_pretrained(pretrained_model_name)
    print("Prepair the tokenizer...")
    tokenizer_name = pretrained_tokenizer_name if pretrained_tokenizer_name else pretrained_model_name
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    print("Set the pipeline.")
    nlp = DistilBertFeatureExtractionPipeline(model=model, tokenizer=tokenizer)
    print("Inference start.")
    # shape: (batch_size, number_of_token, vector_size)
    features = None
    if isinstance(text, str):
        vector = nlp(text, add_special_tokens=False)
        vector = np.array(vector)
        # Take mean of embedding not [CLS] tokens (added by add_special_tokens)
        # https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part9.html
        # features = np.squeeze(vector[:, 0, :])  # for CLS
        features = np.mean(vector, axis=1)
    else:
        features = []
        indexes = [i for i, s in
                   sorted(enumerate(text), key=lambda x: len(x[1]))]
        _text = sorted(text, key=lambda s: len(s))
        for i in tqdm(range(0, len(_text), batch_size)):
            batch = _text[i:i + batch_size]
            vector = nlp(batch, add_special_tokens=False)
            feature = np.array([np.mean(v[0], axis=0) for v in vector])
            # for CLS token
            #vector = nlp(batch, add_special_tokens=True)
            #feature = np.array([v[0][0] for v in vector])
            features.append(feature)

        features = np.vstack(features)
        features = features[np.argsort(indexes)]

    return features


if __name__ == "__main__":
    feature = encode("bandainamco-mirai/distilbert-base-japanese",
                    ["スーパーでリンゴを買ってきた。",
                     "明日は晴れだと思うので、スーパーに行きたい。",
                     "おいしそうなリンゴを買ってきた。",
                     "私は歌手です。",
                     "焼き肉は石川さんが詳しい。"],
                     batch_size=2,
                     pretrained_tokenizer_name="cl-tohoku/bert-base-japanese-whole-word-masking")

    print(feature.shape)
    print(np.max(feature, axis=1))
    X = np.reshape(feature[0, :], (1, -1))
    y = feature[1:, :]

    distance = cosine_similarity(X, y)
    print(distance)
    print(np.argmax(distance))


Loading pretrained model...


Some weights of the model checkpoint at bandainamco-mirai/distilbert-base-japanese were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Prepair the tokenizer...
Set the pipeline.
Inference start.


100%|██████████| 3/3 [00:00<00:00, 12.14it/s]

(5, 768)
[1.99788629 1.80424624 2.09104973 2.06401005 2.08719642]
[[0.98311963 0.99645886 0.98426226 0.99081018]]
1


In [41]:
# from esg_nlp.model.encoder import encode
# from model.encoder import encode

ModuleNotFoundError: ignored

In [70]:
if LANG == "ja":
    # 参考: https://github.com/BandaiNamcoResearchInc/DistilBERT-base-jp/blob/main/docs/GUIDE.md
    model_name = "bandainamco-mirai/distilbert-base-japanese"
    pretrained_tokenizer_name = "cl-tohoku/bert-base-japanese-whole-word-masking"
else:
    model_name = "distilbert-base-uncased"
    pretrained_tokenizer_name = ""

embeddings = encode(model_name,sentences_df["sentence"].values.tolist(),
                    pretrained_tokenizer_name=pretrained_tokenizer_name)

Loading pretrained model...


Some weights of the model checkpoint at bandainamco-mirai/distilbert-base-japanese were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Prepair the tokenizer...
Set the pipeline.
Inference start.


100%|██████████| 47/47 [01:49<00:00,  2.33s/it]


In [71]:
embeddings.shape

(462, 768)

In [72]:
query = encode(model_name, question, pretrained_tokenizer_name=pretrained_tokenizer_name)
query = np.reshape(query, (1, -1))

Loading pretrained model...


Some weights of the model checkpoint at bandainamco-mirai/distilbert-base-japanese were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Prepair the tokenizer...
Set the pipeline.
Inference start.


In [73]:
query.shape

(1, 768)

In [74]:
from sklearn.metrics.pairwise import cosine_similarity


distance = cosine_similarity(query, embeddings)
measured_df = sentences_df.assign(distance=distance.flatten())
measured_df.sort_values("distance", ascending=False, inplace=True)
measured_df.head(10)

,page,order,sentence_order,sentence,length,distance
447,53,6,0,コーポレートデータ 将来予測表明等に関する特記 本資料には、当社(連結子会社を含む)の見通し...,240,0.993665
185,32,11,1,分析・評価 取締役会の構成・運営や監督機能発揮に関するアンケートを実施後、その結果に基づ...,144,0.992093
227,34,19,1,政策保有に関する方針 当社は、政策保有株式について、その保有の意義が認められる場合を除...,219,0.992015
284,38,21,0,atsg は、取り組むべき対策として、組織的管 理策、人的管理策、技術的管理策、物理的管理策...,743,0.991769
7,1,26,0,本統合報告書 2021 ( integrated report 2021 )は、トヨタがめざ...,180,0.991585
461,53,6,14,生産および販売面への影響を含む、自然災害および 感染症の発生・蔓延によるさまざまな影響...,133,0.991502
287,39,21,0,※ 1 人権への負の影響を特定、予防、軽減するために実施されるプロセス 外国人労働者の人...,344,0.991474
231,35,19,0,戦略 トヨタ環境チャレンジ 2050 トヨタは気候変動、水不足、資源枯渇、生物多様性低下な...,716,0.991339
453,53,6,6,当社が営業活動を行っている市場内における法律、規制および政府政策の変更で、特にリコール...,168,0.991285
282,38,19,0,開発、調達、生産、販売、アフターサービスな ど、すべてのプロセスに関わる従業員一人ひとりが仕...,319,0.991118


In [75]:
distance_df = measured_df.groupby(["page", "order"]).median()["distance"].reset_index()
evaluation_df = pd.merge(preprocessed_df, distance_df, how="inner", on=["page", "order"])
evaluation_df.sort_values("distance", ascending=False).head(5)

,page,order,content,preprocessed,length,keyword_match,distance
133,38,21,ATSG\nは、取り組むべき対策として、組織的管\n理策、人的管理策、技術的管理策、物理的管...,atsg は、取り組むべき対策として、組織的管 理策、人的管理策、技術的管理策、物理的管理策...,743,2,0.991769
7,1,26,本統合報告書\n2021\n（\nIntegrated Report 2021\n）は、トヨ...,本統合報告書 2021 ( integrated report 2021 )は、トヨタがめざ...,180,0,0.991585
136,39,21,※\n1 \n人権への負の影響を特定、予防、軽減するために実施されるプロセス\n外国人労働...,※ 1 人権への負の影響を特定、予防、軽減するために実施されるプロセス 外国人労働者の人...,344,1,0.991474
131,38,19,開発、調達、生産、販売、アフターサービスな\nど、すべてのプロセスに関わる従業員一人ひとりが...,開発、調達、生産、販売、アフターサービスな ど、すべてのプロセスに関わる従業員一人ひとりが仕...,319,0,0.991118
99,31,8,基本的な考え方\n トヨタは、持続的な成長と長期安定的な企業価値の向上を経営の重要課題として...,基本的な考え方 トヨタは、持続的な成長と長期安定的な企業価値の向上を経営の重要課題としてい...,178,1,0.991020


In [76]:
evaluation_df["keyword_score"] = evaluation_df["keyword_match"] / len(question_words)
evaluation_df["similarity_score"] = evaluation_df["distance"].clip(lower=0)

In [77]:
evaluation_df.head(5)

,page,order,content,preprocessed,length,keyword_match,distance,keyword_score,similarity_score
0,1,7,カーボンニュートラル実現に向けた取り組み：バッテリー\nEV\n戦略\nトヨタは\n2021...,カーボンニュートラル実現に向けた取り組み:バッテリー ev 戦略 トヨタは 2021 年 1...,337,0,0.988086,0.0,0.988086
1,1,8,カーボンニュートラル実現へトヨタがすべきこと\nオープニング映像が終わり、会場が明かりで照ら...,カーボンニュートラル実現へトヨタがすべきこと オープニング映像が終わり、会場が明かりで照らさ...,324,0,0.987018,0.0,0.987018
2,1,9,限りすぐに、\nCO\n2\nを減らさなければなりません。\n今、私たちは、多様化した世界で...,限りすぐに、 co 2 を減らさなければなりません。 今、私たちは、多様化した世界で、何が正...,364,0,0.986493,0.0,0.986493
3,1,10,ベールを脱いだ５台の\nbZ\n豊田\n本日、皆様に、私たちが準備している新型車をご披露させ...,ベールを脱いだ5台の bz 豊田 本日、皆様に、私たちが準備している新型車をご披露させていた...,658,0,0.985976,0.0,0.985976
4,1,11,航続距離を伸ばすために電池を増やせば、クルマは大きく、重く、高くなります。小さいクルマだから...,航続距離を伸ばすために電池を増やせば、クルマは大きく、重く、高くなります。小さいクルマだから...,359,0,0.986708,0.0,0.986708


In [78]:
evaluation_df["score"] = evaluation_df["keyword_score"] * 0.8 + evaluation_df["similarity_score"] * 0.2
evaluation_df.sort_values("score", ascending=False)

,page,order,content,preprocessed,length,keyword_match,distance,keyword_score,similarity_score,score
116,35,18,トヨタは、金融安定理事会「気候関連財務情報\n開示タスクフォース（\nTCFD\n）」の提言...,トヨタは、金融安定理事会「気候関連財務情報 開示タスクフォース( tcfd )」の提言に 2...,591,6,0.990131,0.6,0.990131,0.678026
117,35,19,戦略\nトヨタ環境チャレンジ\n2050\n トヨタは気候変動、水不足、資源枯渇、生物多様性...,戦略 トヨタ環境チャレンジ 2050 トヨタは気候変動、水不足、資源枯渇、生物多様性低下な...,1093,4,0.982915,0.4,0.982915,0.516583
120,36,17,販車も含めた\nCO\n2\n排出削減に向け、技術の選択肢\nを広げることも重要であると考え...,販車も含めた co 2 排出削減に向け、技術の選択肢 を広げることも重要であると考えています...,845,3,0.989178,0.3,0.989178,0.437836
118,35,20,重要度の高いリスク／機会と対応\nリスク機会トヨタの対応気候シナリオとの関係現行シナリオ対策...,重要度の高いリスク/機会と対応 リスク機会トヨタの対応気候シナリオとの関係現行シナリオ対策強...,622,3,0.978638,0.3,0.978638,0.435728
133,38,21,ATSG\nは、取り組むべき対策として、組織的管\n理策、人的管理策、技術的管理策、物理的管...,atsg は、取り組むべき対策として、組織的管 理策、人的管理策、技術的管理策、物理的管理策...,743,2,0.991769,0.2,0.991769,0.358354
...,...,...,...,...,...,...,...,...,...,...
28,7,16,5. \nお寿司のリードタイムを目指せ\n例えば、お寿司の世界ね。\nちゃんと目の前で握って...,5. お寿司のリードタイムを目指せ 例えば、お寿司の世界ね。 ちゃんと目の前で握ってくれる...,209,0,0.965898,0.0,0.965898,0.193180
182,51,22,6.68.56.26.9\n8.3\n■■\n \n親会社の所有者に帰属する当期利益\n \...,6.68.56.26.9 8.3 ■■ 親会社の所有者に帰属する当期利益 当期利...,108,0,0.964184,0.0,0.964184,0.192837
112,34,17,2017/32018/32019/32020/32021/31\n株当たり配当額（円）\n2...,2017/32018/32019/32020/32021/31 株当たり配当額(円) 210...,400,0,0.963421,0.0,0.963421,0.192684
147,41,10,女性比率［％］男女別平均勤続年数［年］採用正社員管理職経営幹部\nトヨタ自動車（日本）\n2...,女性比率[%]男女別平均勤続年数[年]採用正社員管理職経営幹部 トヨタ自動車(日本) 26....,762,0,0.962506,0.0,0.962506,0.192501


In [79]:
np.round(evaluation_df.nlargest(1, "score")["score"].item(), 2)

0.68